In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm

# Using pandas to load the CSV file into a DataFrame.
# Inspecting the first few rows to understand the structure and content.
df = pd.read_csv('stop_tb_data2.csv')
print(df.head())



ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Checking the number of missing values per column
missing_values_count = df.isnull().sum()

# Checking the percentage of missing values per column
missing_values_percentage = df.isnull().mean() * 100

# Print results
print(missing_values_count)
print(missing_values_percentage)


In [3]:
# Dropping columns with more than 60% missing values


df = df.dropna(thresh=len(df) * 0.6, axis=1)

# Filling missing values with median for numerical columns
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())


In [ ]:
# Checking the number of missing values per column again
missing_values_count = df.isnull().sum()

# Checking the percentage of missing values per column again
missing_values_percentage = df.isnull().mean() * 100

# Print results
print(missing_values_count)
print(missing_values_percentage)


In [ ]:
# Display the first few rows of the dataframe
print(df.head())

# Display a summary of the dataframe to check data types and non-null counts
print(df.info())


In [ ]:
# Drop non-numeric or irrelevant columns
df_filtered = df.drop(columns=['country_name', 'iso3_code', 'g_whoregion'])

# Convert remaining categorical columns to numeric using one-hot encoding
df_numeric = pd.get_dummies(df_filtered, drop_first=True)

# Calculate the correlation matrix
correlation_matrix = df_numeric.corr()

# Create a heatmap to visualize the correlation matrix
plt.figure(figsize=(20, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Load the WHO dataset
who_df = pd.read_csv('tb_incidence_data_zambia.csv')

# Rename 'Year' to 'year' in who_df 
who_df.rename(columns={'Year': 'year'}, inplace=True)

# Now merge again
merged_df = pd.merge(df, who_df, on='year', how='inner')

# Display the merged DataFrame
print(merged_df.head())



In [8]:
# Export the merged DataFrame to a CSV file
merged_df.to_csv('merged_tb_data.csv', index=False)

In [ ]:
print(merged_df.describe())

In [ ]:
# Drop columns that start with 'e_' or 'estimated'
columns_to_drop = [col for col in merged_df.columns if col.startswith('e_') or col.startswith('estimated')]
merged_df = merged_df.drop(columns=columns_to_drop)

# Convert categorical variables to numerical using one-hot encoding
merged_df_encoded = pd.get_dummies(merged_df, drop_first=True)

# List of target variables
target_vars = ['TB cases per 100 000', 'Number of incident tuberculosis cases']

# Compute the correlation matrix for all variables against the target variables
correlation_matrix = merged_df_encoded.corr()

# Extract correlations with the target variables
correlation_with_tb = correlation_matrix[target_vars].dropna()

# Display the correlation matrix for inspection
print(correlation_with_tb)

# Generate a heatmap for the correlation matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(22, 18))
sns.heatmap(correlation_with_tb, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Correlation Matrix with TB cases per 100,000 and Incident TB Cases')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Load the data
# Ensure 'year' is the index
merged_df_encoded['year'] = pd.to_datetime(merged_df_encoded['year'], format='%Y')
merged_df_encoded.set_index('year', inplace=True)

# Select the two target variables
tb_cases = merged_df_encoded['TB cases per 100 000']
tb_incidence = merged_df_encoded['Number of incident tuberculosis cases']

# Split the data into training and testing sets (80% train, 20% test)
train_size = int(len(tb_cases) * 0.8)
train_tb_cases, test_tb_cases = tb_cases[:train_size], tb_cases[train_size:]
train_tb_incidence, test_tb_incidence = tb_incidence[:train_size], tb_incidence[train_size:]

# Function to build and evaluate ARIMA model
def arima_forecast(train, test, order=(1,1,1)):
    # Fit ARIMA model
    model = ARIMA(train, order=order)
    model_fit = model.fit()

    # Make predictions
    predictions = model_fit.forecast(steps=len(test))
    
    # Evaluate model performance
    rmse = np.sqrt(mean_squared_error(test, predictions))
    print(f'RMSE: {rmse}')
    
    # Plot the results
    plt.figure(figsize=(10,6))
    plt.plot(train.index, train, label='Train Data')
    plt.plot(test.index, test, label='Test Data', color='orange')
    plt.plot(test.index, predictions, label='Predictions', color='green')
    plt.title('ARIMA Forecast')
    plt.legend()
    plt.show()
    
    return model_fit, predictions

# Build ARIMA model for 'TB cases per 100 000'
print("Model for 'TB cases per 100 000'")
arima_tb_cases, tb_cases_predictions = arima_forecast(train_tb_cases, test_tb_cases)

# Build ARIMA model for 'Number of incident tuberculosis cases'
print("\nModel for 'Number of incident tuberculosis cases'")
arima_tb_incidence, tb_incidence_predictions = arima_forecast(train_tb_incidence, test_tb_incidence)

In [ ]:
# Load the data
merged_df_encoded['year'] = pd.to_datetime(merged_df_encoded['year'], format='%Y')
merged_df_encoded.set_index('year', inplace=True)

# Select the target and exogenous variables
tb_cases = merged_df_encoded['TB cases per 100 000']
tb_incidence = merged_df_encoded['Number of incident tuberculosis cases']
hiv_rates = merged_df_encoded['HIV rates']  # Example exogenous variable
population = merged_df_encoded['Population']  # Another exogenous variable

# Combine exogenous variables into a matrix (X)
exog = merged_df_encoded[['HIV rates', 'Population']]

# Split the data into training and testing sets (80% train, 20% test)
train_size = int(len(tb_cases) * 0.8)
train_tb_cases, test_tb_cases = tb_cases[:train_size], tb_cases[train_size:]
train_tb_incidence, test_tb_incidence = tb_incidence[:train_size], tb_incidence[train_size:]

# Also split exogenous variables for training and testing
train_exog, test_exog = exog[:train_size], exog[train_size:]

# 1. Function to build and evaluate ARIMA model with auto ARIMA
def arima_forecast_with_auto(train, test, exog_train=None, exog_test=None):
    # Use auto_arima to find the best order
    model = pm.auto_arima(train, exogenous=exog_train, seasonal=False, trace=True,
                          suppress_warnings=True, stepwise=True)
    print(f'Best ARIMA order: {model.order}')
    
    # Fit ARIMA model
    model_fit = model.fit(train, exogenous=exog_train)

    # Make predictions
    predictions = model_fit.predict(n_periods=len(test), exogenous=exog_test)
    
    # Evaluate model performance
    rmse = np.sqrt(mean_squared_error(test, predictions))
    print(f'RMSE: {rmse}')
    
    # Plot the results
    plt.figure(figsize=(10,6))
    plt.plot(train.index, train, label='Train Data')
    plt.plot(test.index, test, label='Test Data', color='orange')
    plt.plot(test.index, predictions, label='Predictions', color='green')
    plt.title('ARIMA Forecast with Exogenous Variables')
    plt.legend()
    plt.show()
    
    return model_fit, predictions

# 2. Function for model diagnostics
def model_diagnostics(model_fit):
    residuals = model_fit.resid()
    
    # Plot residuals
    plt.figure(figsize=(10,6))
    plt.subplot(211)
    plt.plot(residuals)
    plt.title('Residuals of the ARIMA Model')
    
    # Plot ACF and PACF of the residuals
    plt.subplot(212)
    plot_acf(residuals, ax=plt.gca(), lags=20)
    plt.show()
    
    # Perform statistical test to ensure residuals are white noise
    ljung_box_test = sm.stats.acorr_ljungbox(residuals, lags=[20], return_df=True)
    print("Ljung-Box Test p-value:", ljung_box_test["lb_pvalue"].values)
    
    if ljung_box_test["lb_pvalue"].values[0] > 0.05:
        print("Residuals resemble white noise.")
    else:
        print("Residuals do not resemble white noise.")

# 3. Build ARIMA model for 'TB cases per 100 000' with exogenous variables
print("Model for 'TB cases per 100 000' with exogenous variables")
arima_tb_cases_exog, tb_cases_predictions_exog = arima_forecast_with_auto(train_tb_cases, test_tb_cases, exog_train=train_exog, exog_test=test_exog)

# Model diagnostics
model_diagnostics(arima_tb_cases_exog)

# 4. Build ARIMA model for 'Number of incident tuberculosis cases' with exogenous variables
print("\nModel for 'Number of incident tuberculosis cases' with exogenous variables")
arima_tb_incidence_exog, tb_incidence_predictions_exog = arima_forecast_with_auto(train_tb_incidence, test_tb_incidence, exog_train=train_exog, exog_test=test_exog)

# Model diagnostics
model_diagnostics(arima_tb_incidence_exog)